In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
import reach_tools

from datetime import datetime, timedelta
import time
import requests
import pandas as pd

from arcgis.gis import GIS

import config

%load_ext autoreload
%autoreload 2

In [214]:
%%time
reach = reach_tools.Reach.get_from_aw(3066)

CPU times: user 29.9 ms, sys: 2.56 ms, total: 32.5 ms
Wall time: 305 ms


In [215]:
self = reach

for rng in self._reach_json['guagesummary']['ranges']:
    print(f"gauge_{rng['range_min'].lower()}", float(rng['gauge_min']))
    print(f"gauge_{rng['range_max'].lower()}", float(rng['gauge_max']))
    setattr(self, f"gauge_{rng['range_min'].lower()}", float(rng['gauge_min']))
    setattr(self, f"gauge_{rng['range_max'].lower()}", float(rng['gauge_max']))

gauge_r_lst = [self.gauge_r1, self.gauge_r2, self.gauge_r3, self.gauge_r4, self.gauge_r5, self.gauge_r6,
                           self.gauge_r7, self.gauge_r8, self.gauge_r9]
gauge_r_lst = [val for val in gauge_r_lst if val is not None]
gauge_r_lst

gauge_r0 500.0
gauge_r5 700.0
gauge_r4 700.0
gauge_r6 1100.0
gauge_r5 1100.0
gauge_r9 1500.0


[700.0, 1100.0, 1100.0, 1500.0]

In [217]:
print(reach.reach_search)
print(f'Gauge ID: {reach.gauge_id}')
print(f'Current Observation: {reach.gauge_observation} {reach.gauge_units}')
print(f'Runnable: {reach.gauge_runnable}')
print(f'Flow Min: {reach.gauge_min} {reach.gauge_units}')
print(f'Flow Max: {reach.gauge_max} {reach.gauge_units}')
print(f'Gauge Stage: {reach.gauge_stage}')

Canyon Creek (Lewis River trib.) 2 - Fly Creek to Merwin Reservoir
Gauge ID: 35665
Current Observation: 277.0 cfs
Runnable: False
Flow Min: 500.0 cfs
Flow Max: 1500.0 cfs
Gauge Stage: too low


In [3]:
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)
gis

GIS @ https://knu2xs.maps.arcgis.com version:7.1

In [4]:
reach_centroid_url = 'https://services3.arcgis.com/p9emohPzcQpdUrEc/arcgis/rest/services/Reach_Centroid/FeatureServer/0'

reach_centroid_lyr = reach_tools.ReachFeatureLayer.from_url(gis, reach_centroid_url)
reach_centroid_lyr

<ReachFeatureLayer url:"https://services3.arcgis.com/p9emohPzcQpdUrEc/arcgis/rest/services/Reach_Centroid/FeatureServer/0">

In [12]:
self = reach_tools.Reach.get_from_aw(3066)

endTime = datetime.utcnow()
startTime = endTime - timedelta(hours=24)

endTimeStamp = round(endTime.timestamp())
startTimeStamp = round(startTime.timestamp())

url = f'http://www.americanwhitewater.org/api/gauge/{self.gauge_id}/flows/{self.gauge_metric}'

params = {
    'from': startTimeStamp,
    'to': endTimeStamp,
    'resolution': 1
}

resp = requests.get(url, params=params)

df = pd.DataFrame.from_dict(resp.json())
df.updated = df.updated.apply(lambda val: datetime.fromtimestamp(round(float(val))))

df

,data,gauge_id,id,metric,nv,reading,updated
0,None,35665,153180203142115,2,1556242176,280,2019-04-25 18:29:36
1,None,35665,153180203142565,2,1556245760,274,2019-04-25 19:29:20
2,None,35665,153180203143015,2,1556249344,274,2019-04-25 20:29:04
3,None,35665,153180203143465,2,1556252928,283,2019-04-25 21:28:48
4,None,35665,153180203143915,2,1556256512,270,2019-04-25 22:28:32
5,None,35665,153180203144365,2,1556260096,280,2019-04-25 23:28:16
6,None,35665,153180203144815,2,1556263680,274,2019-04-26 00:28:00
7,None,35665,153180203145265,2,1556267392,270,2019-04-26 01:29:52
8,None,35665,153180203145715,2,1556270976,274,2019-04-26 02:29:36
9,None,35665,153180203146165,2,1556274560,267,2019-04-26 03:29:20


In [219]:
last_obs_row = df.loc[df['updated'].idxmax()]
last_obs_row

data                       None
gauge_id                  35665
id              153180203141215
metric                        2
nv                   1556234880
reading                     277
updated     2019-04-25 16:28:00
Name: 20, dtype: object

In [220]:
last_obs = float(last_obs_row.reading)
last_obs

277.0

In [137]:
oid_resp = reach_centroid_lyr.query(return_ids_only=True)
oids = ','.join([str(v) for v in oid_resp['objectIds']])
gauge_fields = ['reach_id', 'gauge_max', 'gauge_min', 'gauge_observation', 'gauge_runnable', 'gauge_r0', 'gauge_r1', 'gauge_r2', 'gauge_r3', 'gauge_r4', 'gauge_r5', 'gauge_r6', 'gauge_r7', 
                'gauge_r8', 'gauge_r9', 'gauge_id', 'gauge_units', 'gauge_metric']
reach_fs = reach_centroid_lyr.query(object_ids=oids, out_fields=gauge_fields, return_geometry=False)
reach_fs

<FeatureSet> 5764 features

In [223]:
false_start_idx = 600
for idx, feature in enumerate(reach_fs.features[false_start_idx:]):
    reach = reach_tools.Reach.get_from_aw(feature.attributes['reach_id'])
    for field in gauge_fields:
        feature.attributes[field] = getattr(reach, field)
    resp = reach_centroid_lyr.edit_features(updates=[feature])
    if feat.attributes['gauge_id']:
        has_gauge = 'has a gauge'
    else:
        has_gauge = 'does NOT have a gauge'
    if resp['updateResults'][0]['success'] is True:
        log_msg = f"MSG: (idx-{false_start_idx + idx}) Reach ID {feature.attributes['reach_id']} Object ID {feature.attributes['OBJECTID']} gauge information was successfully updated and {has_gauge}."
    else:
        log_msg = f"ERROR: (idx-{false_start_idx + idx}) Reach ID {feature.attributes['reach_id']} Object ID {feature.attributes['OBJECTID']} gauge information was NOT successfully updated and {has_gauge}."
    print(log_msg)

MSG: (idx-600) Reach ID 580 Object ID 619 gauge information was successfully updated and has a gauge.
MSG: (idx-601) Reach ID 581 Object ID 620 gauge information was successfully updated and has a gauge.
MSG: (idx-602) Reach ID 582 Object ID 621 gauge information was successfully updated and has a gauge.
MSG: (idx-603) Reach ID 583 Object ID 622 gauge information was successfully updated and has a gauge.
MSG: (idx-604) Reach ID 584 Object ID 623 gauge information was successfully updated and has a gauge.
MSG: (idx-605) Reach ID 585 Object ID 624 gauge information was successfully updated and has a gauge.
MSG: (idx-606) Reach ID 586 Object ID 625 gauge information was successfully updated and has a gauge.
MSG: (idx-607) Reach ID 587 Object ID 626 gauge information was successfully updated and has a gauge.
MSG: (idx-608) Reach ID 588 Object ID 627 gauge information was successfully updated and has a gauge.
MSG: (idx-609) Reach ID 589 Object ID 628 gauge information was successfully updat

KeyboardInterrupt: 

In [207]:
feat.attributes['gauge_id']

{'abstract': 'This is a micro creek that runs through the Lost Valley Campground. The creek is fast class II from 0.6 miles above the campground to the Buffalo River. Above the class II section are several class III-V drops. With enough water to be runnable, your put-in destination is the the Natural Bridge (Cave) that the creek flows through. Park and hike up the trail to the cave. There are a couple nice drops above the cave, but will take a lot of water to make the entrance drop go. Slide in at the top of...',
 'agency': 'Buffalo National River Park',
 'centroid': {'x': -93.37845230102539,
  'y': 36.0115489959717,
  'spatialReference': {'wkid': 4326}},
 'description': 'This is a micro creek that runs through the Lost Valley Campground. The creek is fast class II from 0.6 miles above the campground to the Buffalo River. Above the class II section are several class III-V drops. With enough water to be runnable, your put-in destination is the the Natural Bridge (Cave) that the creek fl